# West Nile Virus Presence Prediction
### Methods
This part mainly contains some descriptive part to see the relevance of the features to be used. 
* Base Models
* OverSampling

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_weather = pd.read_csv('weather.csv')
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [5]:
df_weather['Water1'].value_counts()

M    2944
Name: Water1, dtype: int64

In [6]:
df_weather['SnowFall'][df_weather['SnowFall']!='0.0'].value_counts()

M      1472
  T      12
0.1       1
Name: SnowFall, dtype: int64

##### Dropping 'Depart', 'Sunrise', 'sunset', 'depth', 'codesum', 'Water1', 'SnowFall'

In [7]:
df_weather = df_weather.drop(['Depart', 'Sunrise', 'Sunset', 'Depth', 'CodeSum', 'Water1' ,'SnowFall'] , axis=1)

In [8]:
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,51,56,0,2,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,51,57,0,3,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,42,47,14,0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,42,47,13,0,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,40,48,9,0,0.00,29.39,30.12,11.7,7,11.9


In [9]:
df_weather = df_weather.replace('M', -9999)
df_weather = df_weather.replace('  T', -9999)
#df_weather.dtypes
#df.loc[~df['Value'].astype(str).str.isdigit()]
#df_weather.loc[~df_weather.astype(str).isdigit()]

In [11]:
df_weather.isnull().describe()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
count,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944,2944


In [12]:
indices = ['Station','Date'] 
otherIndices = df_weather.columns.difference(indices)

In [13]:
df_weather[otherIndices] = df_weather[otherIndices].astype(float)

In [14]:
df_weather['joinID'] = df_weather['Date'].astype(str)+df_weather['Station'].astype(str)
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,joinID
0,1,2007-05-01,83.0,50.0,67.0,51.0,56.0,0.0,2.0,0.0,29.10,29.82,1.7,27.0,9.2,2007-05-011
1,2,2007-05-01,84.0,52.0,68.0,51.0,57.0,0.0,3.0,0.0,29.18,29.82,2.7,25.0,9.6,2007-05-012
2,1,2007-05-02,59.0,42.0,51.0,42.0,47.0,14.0,0.0,0.0,29.38,30.09,13.0,4.0,13.4,2007-05-021
3,2,2007-05-02,60.0,43.0,52.0,42.0,47.0,13.0,0.0,0.0,29.44,30.08,13.3,2.0,13.4,2007-05-022
4,1,2007-05-03,66.0,46.0,56.0,40.0,48.0,9.0,0.0,0.0,29.39,30.12,11.7,7.0,11.9,2007-05-031


In [15]:
# load train and test data
df_train = pd.read_csv('train.csv')
df_train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


#### Location Mapping

In [16]:
# find the nearest station
def mapStation(loc):
    station1 = np.array([41.995,-87.933])
    station2 = np.array([41.786,-87.752])
    testStation = np.array([loc[0],loc[1]])
    distance1 = np.linalg.norm(station1-testStation)
    distance2 = np.linalg.norm(station2-testStation)
    #print(distance1,distance2)
    if distance1 < distance2 : return 1
    return 2

In [17]:
#train set
lat_lon = df_train[['Latitude','Longitude']].copy()
df_train['Station'] = lat_lon.apply(mapStation, axis=1)

In [18]:
df_train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Station
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,1
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,1
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,1


In [19]:
# drop Addresses, NumMosquitos
df_train = df_train[['Date','Station','Species','WnvPresent']]
df_train['joinID'] = df_train['Date'].astype(str)+df_train['Station'].astype(str)

In [20]:
df_train.head()

,Date,Station,Species,WnvPresent,joinID
0,2007-05-29,1,CULEX PIPIENS/RESTUANS,0,2007-05-291
1,2007-05-29,1,CULEX RESTUANS,0,2007-05-291
2,2007-05-29,1,CULEX RESTUANS,0,2007-05-291
3,2007-05-29,1,CULEX PIPIENS/RESTUANS,0,2007-05-291
4,2007-05-29,1,CULEX RESTUANS,0,2007-05-291


In [21]:
df_train = df_train.merge(df_weather, on='joinID', how='inner')
df_train.head()

,Date_x,Station_x,Species,WnvPresent,joinID,Station_y,Date_y,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,1,CULEX PIPIENS/RESTUANS,0,2007-05-291,1,2007-05-29,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5
1,2007-05-29,1,CULEX RESTUANS,0,2007-05-291,1,2007-05-29,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5
2,2007-05-29,1,CULEX RESTUANS,0,2007-05-291,1,2007-05-29,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5
3,2007-05-29,1,CULEX PIPIENS/RESTUANS,0,2007-05-291,1,2007-05-29,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5
4,2007-05-29,1,CULEX RESTUANS,0,2007-05-291,1,2007-05-29,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5


In [22]:
features = ['Species','Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'Heat', 'Cool','PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir','AvgSpeed']
label = 'WnvPresent'
X = df_train[features]
X = X.replace(-9999, np.nan)
X = X.fillna(X.mean())

In [23]:
X = pd.get_dummies(X)

In [24]:
X.head()

,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5,0,0,1,0,0,0,0
1,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5,0,0,0,1,0,0,0
2,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5,0,0,0,1,0,0,0
3,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5,0,0,1,0,0,0,0
4,88.0,60.0,74.0,58.0,65.0,0.0,9.0,0.0,29.39,30.11,5.8,18.0,6.5,0,0,0,1,0,0,0


#### Training Models

In [25]:
dataX = X.values 
dataY = df_train[label].values

In [31]:
# train test split
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(dataX, dataY, test_size=0.3, random_state=42)

In [27]:
from sklearn.preprocessing import StandardScaler
z = StandardScaler()
trainX = z.fit_transform(trainX)
validX = z.transform(validX)

In [32]:
trainX.shape, validX.shape, trainY.shape, validY.shape

((7354, 20), (3152, 20), (7354,), (3152,))

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix 
lr = LogisticRegression()

lr.fit(trainX[:,13:], trainY)
y_pred = lr.predict(validX[:,13:])

print('Training: \n', lr.score(trainX[:,13:], trainY))
print(classification_report(trainY, lr.predict(trainX[:,13:])))
print(confusion_matrix(trainY, lr.predict(trainX[:,13:])))

print('Validation: \n', lr.score(validX[:,13:], validY))
print(classification_report(validY, y_pred))
print(confusion_matrix(validY, y_pred))

Training: 
 0.9471036170791406
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6965
          1       0.00      0.00      0.00       389

avg / total       0.90      0.95      0.92      7354

[[6965    0]
 [ 389    0]]
Validation: 
 0.9486040609137056
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      2990
          1       0.00      0.00      0.00       162

avg / total       0.90      0.95      0.92      3152

[[2990    0]
 [ 162    0]]


C:\Users\10257559\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix 
nb = MultinomialNB(alpha=0.5)

nb.fit(trainX, trainY)


print('Training: \n', nb.score(trainX, trainY))
print(classification_report(trainY, nb.predict(trainX)))
print(confusion_matrix(trainY, nb.predict(trainX)))

print('Validation: \n', nb.score(validX, validY))
print(classification_report(validY, nb.predict(validX)))
print(confusion_matrix(validY, nb.predict(validX)))

Training: 
 0.9180038074517269
             precision    recall  f1-score   support

          0       0.95      0.97      0.96      6965
          1       0.03      0.02      0.02       389

avg / total       0.90      0.92      0.91      7354

[[6745  220]
 [ 383    6]]
Validation: 
 0.9197335025380711
             precision    recall  f1-score   support

          0       0.95      0.97      0.96      2990
          1       0.05      0.03      0.04       162

avg / total       0.90      0.92      0.91      3152

[[2894   96]
 [ 157    5]]


In [35]:
from sklearn.ensemble import GradientBoostingClassifier 
gbc = GradientBoostingClassifier()

gbc.fit(trainX, trainY)


print('Training: \n', gbc.score(trainX, trainY))
print(classification_report(trainY, gbc.predict(trainX)))
print(confusion_matrix(trainY,gbc.predict(trainX)))

print('Validation: \n', gbc.score(validX, validY))
print(classification_report(validY, gbc.predict(validX)))
print(confusion_matrix(validY, gbc.predict(validX)))

Training: 
 0.9483274408485178
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6965
          1       0.67      0.05      0.09       389

avg / total       0.93      0.95      0.93      7354

[[6956    9]
 [ 371   18]]
Validation: 
 0.9486040609137056
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      2990
          1       0.50      0.04      0.07       162

avg / total       0.93      0.95      0.93      3152

[[2984    6]
 [ 156    6]]


### Model Performances are very poor. 
#### Let's try by balancing the classes (using oversampling)

In [36]:
from imblearn.over_sampling import RandomOverSampler
os = RandomOverSampler()
X_resampled, y_resampled = os.fit_sample(dataX, dataY)


trainX, validX, trainY, validY = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


lr = LogisticRegression()

lr.fit(trainX, trainY)


print('Training: \n', lr.score(trainX, trainY))
print(classification_report(trainY, lr.predict(trainX)))
print(confusion_matrix(trainY, lr.predict(trainX)))

print('Validation: \n', lr.score(validX, validY))
print(classification_report(validY,lr.predict(validX)))
print(confusion_matrix(validY, lr.predict(validX)))



Training: 
 0.6545885054172347
             precision    recall  f1-score   support

          0       0.69      0.55      0.61      6940
          1       0.63      0.76      0.69      6997

avg / total       0.66      0.65      0.65     13937

[[3831 3109]
 [1705 5292]]
Validation: 
 0.6517662816005357
             precision    recall  f1-score   support

          0       0.69      0.57      0.62      3015
          1       0.63      0.74      0.68      2958

avg / total       0.66      0.65      0.65      5973

[[1709 1306]
 [ 774 2184]]


In [37]:
lr = LogisticRegression(penalty='l1', C=0.1)
lr.fit(trainX, trainY)


print('Training: \n', lr.score(trainX, trainY))
print(classification_report(trainY, lr.predict(trainX)))
print(confusion_matrix(trainY, lr.predict(trainX)))

print('Validation: \n', lr.score(validX, validY))
print(classification_report(validY,lr.predict(validX)))
print(confusion_matrix(validY, lr.predict(validX)))

Training: 
 0.6479873717442778
             precision    recall  f1-score   support

          0       0.68      0.56      0.61      6940
          1       0.63      0.74      0.68      6997

avg / total       0.65      0.65      0.64     13937

[[3855 3085]
 [1821 5176]]
Validation: 
 0.646911099949774
             precision    recall  f1-score   support

          0       0.68      0.57      0.62      3015
          1       0.62      0.72      0.67      2958

avg / total       0.65      0.65      0.64      5973

[[1724 1291]
 [ 818 2140]]


## Performance Improved: Still not good !!!!

#### Let's try to improve the performance. Part 2